In [164]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer

### 1.Import data

In [165]:
data = pd.read_csv('../data/loan/train.csv')

data = data.drop('Loan_ID', axis=1)
data['Dependents'] = data['Dependents'].replace('3+', '3')

X, y = data.drop('Loan_Status', axis=1), data.pop('Loan_Status')

X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y)

In [166]:
""" from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted

class DropCols(BaseEstimator, TransformerMixin):
    def __init__(self):
        return 
        #self.cols = cols
        
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        self.X = X.copy()
        return self.X.drop(self.X.columns, axis=1)
        #print(self.X.columns)
        return X
 """

' from sklearn.base import BaseEstimator, TransformerMixin\nfrom sklearn.utils.validation import check_is_fitted\n\nclass DropCols(BaseEstimator, TransformerMixin):\n    def __init__(self):\n        return \n        #self.cols = cols\n        \n    def fit(self, X, y=None):\n        return self\n\n    def transform(self, X, y=None):\n        self.X = X.copy()\n        return self.X.drop(self.X.columns, axis=1)\n        #print(self.X.columns)\n        return X\n '

In [167]:
ct_fill = make_column_transformer(
    (SimpleImputer(strategy="most_frequent"),make_column_selector(dtype_include=object)),
    (OrdinalEncoder(categories=[["Not Graduate", "Graduate"]]), ["Education"]),
    (SimpleImputer(strategy="mean"), make_column_selector(dtype_include="number")),
    (OrdinalEncoder(categories=[["Rural", "Semiurban", "Urban"]]), ["Property_Area"]),
    (OneHotEncoder(handle_unknown='ignore'), ["Gender", "Married"]),
    remainder='passthrough'
)

mypipe = make_pipeline(ct_fill)

In [168]:
X_train = pd.DataFrame(mypipe.fit_transform(X_train)).iloc[:, 6:].astype(float)
X_test = pd.DataFrame(mypipe.transform(X_test)).iloc[:, 6:].astype(float)

y_train = y_train.map({'Y':1, 'N':0})
y_test = y_test.map({'Y':1, 'N':0})

### 2. Install LightGBM

In [169]:
#%pip install lightgbm
from lightgbm import LGBMClassifier

### 3. Build a Model

In [170]:
model = LGBMClassifier()


### 3. Evaluate model

In [171]:
model.fit(X_train, y_train)
model.score(X_train, y_train), model.score(X_test, y_test)

(0.9891304347826086, 0.7792207792207793)

### 4. Hyperparameter Tunning

In [191]:
model = LGBMClassifier(learning_rate=0.001, n_estimators=500, objective='binary')
model.fit(X_train, y_train)
model.score(X_train, y_train), model.score(X_test, y_test)

(0.8130434782608695, 0.7987012987012987)